# Haskell Kōan: probabilstic programming in < 100 LoC

Let us begin with our incantations to GHC:

In [1]:
{-# LANGUAGE GADTs #-}
import Control.Monad (ap, replicateM)
import System.Random (getStdGen, getStdRandom, randomR)
import qualified Data.Map as M

First, we create a new monad called `P`, for *probability*.
It supports only two operations:

- `Return` converts a pure value into a `P` value,
- `Sample` samples from a uniform distribution over [0, 1].

We encode the monad in CPS style to get a monad instance for-free. For more
modularity, use the free monad technique.

In [2]:
data P a where
  Return :: a -> P a -- ^ lift a pure value
  Sample :: (Float -> P a) -> P a -- ^ use a float uniformly sampled from [0, 1] 
  
instance Functor P where
  fmap f (Return x) = Return (f x)
  fmap f (Sample rand2pa) = Sample $ \r -> fmap f (rand2pa r)
  
instance Monad P where
  return = Return
  Return a >>= a2pb = a2pb a
  Sample r2pa >>= a2pb = Sample $ \r -> r2pa r >>= a2pb

instance Applicative P where
  pure = return
  (<*>) = ap
  
sample01 :: P Float
sample01 = Sample Return

Now that we have a way to build expressions in this language, let's build interesting objects from this.
We start with a coin, and then we use the coin to approximate a normal distribution by adding many coins.

In [3]:
-- | 'coin b' returns 1 with probability p, 0 with probability (1 - p) 
coin :: Num a => Float -> P a
coin p = do
  r <- sample01
  return $ if r < p then 1 else 0
  
-- | approximate a normal distribution using the central limit theorem by adding many
-- uniformly distributed {0, 1} variables. Here, mean = 1
normal :: (Fractional a, Num a) => P a 
normal = do
  as <- replicateM 10 (coin 0.5)
  return $ sum as / 5

We need a way to run our computation and plot its values, so let's build that capability

In [4]:
-- | Sample a random value from a `P a`
runP :: P a -> IO a
runP (Return a) = return a
runP (Sample rand2pa) = do
  r <- getStdRandom $ randomR (0, 1)
  runP (rand2pa r)

-- | Sample `n` random values from a `P a`
runsP :: Int -> P a -> IO [a]
runsP n p = replicateM n (runP p)

Let's also write a tiny utility to plot these values onto the command line with fancy ASCII art.
Notice that this will take us more code than everything we have done so far

In [5]:
-- | List of characters that represent sparklines
sparkchars :: String
sparkchars = "_▁▂▃▄▅▆▇█"

-- Convert an int to a sparkline character
num2spark :: RealFrac a => a -- ^ Max value
  -> a -- ^ Current value
  -> Char
num2spark maxv curv =
   sparkchars !!
     (floor $ (curv / maxv) * (fromIntegral (length sparkchars - 1)))

-- | Print sparklines with title
printvals :: RealFrac a => String -> [a] -> IO ()
printvals title vs = do 
  let maxv = if null vs then 0 else maximum vs
  putStrLn $ title ++ " " ++ map (num2spark maxv) vs
  
  -- | Create a histogram from values.
histogram :: RealFrac a
          => String -- ^ title
          -> Int -- ^ number of buckets
          -> [a] -- values
          -> IO ()
histogram title nbuckets as = do
        let minv = minimum as
        let maxv = maximum as
        let perbucket = (maxv - minv) / (fromIntegral nbuckets)
        let bucket v = floor (v / perbucket)
        let bucketed = foldl (\m v -> M.insertWith (+) (bucket v) 1 m) mempty as
        printvals title $ map snd . M.toList $ bucketed

We now have all the pieces we need to experiment!

In [6]:
runsP 20 (coin 0) >>=  printvals "bias 0"
runsP 20 (coin 0.2) >>=  printvals "bias 0.2"
runsP 20 (coin 0.5) >>=  printvals "bias 0.5"
runsP 20 (coin 0.8) >>=  printvals "bias 0.8"
runsP 20 (coin 1) >>=  printvals "bias 1"
runsP 1000 normal >>= histogram "normal" 10 

bias 0 ____________________

bias 0.2 ______█________█____

bias 0.5 __██__████_█__█_____

bias 0.8 __███_█████████████_

bias 1 ████████████████████

normal __▂▃█▁___

So, we can create coins, gaussians, and whatever we want _from the building block of a uniform distribution_.
However, we still don't have a way to take samples from _arbitrary distributions_.
What if we want to sample from a distribution of the form `p(x) = x^{-2}`, or `p(x) = sin(x)`?
Enter MCMC (markov-chain-monte-carlo), a class of powerful techniques that allow us to sample from any choice of distribution.

The part of this that really tickles my fancy is that the MCMC algorithm we are going to implement
(metropolis-hastings) _is described within our `P` monad_.

That is, we have a constructive proof that given the ability to sample from a uniform distribution,
we can sample from _any_ (computable) distribution. I find this quite fact
really marvellous -- it is rare that mathematical objects behave this well with respect
to computation.

In [10]:
-- | take a sampler and create a new probabilstic value, which samples from the original
-- based on the score values (Metropolis-Hastings)
mhStep :: (a -> Float) -- ^ score function
   -> (a -> P a) -- ^ proposal: given a value, provide values around that value
   -> a -- ^ current value
   -> P a -- ^ next value
mhStep scoring proposer a = do
   a' <- proposer a
   let accept = scoring a' / scoring a
   u <- sample01
   return $ if u < accept then a' else a

-- | Take N steps of metropolois-hastings
mhSteps :: Int -> (a -> Float) -> (a -> P a) -> a -> P a
mhSteps 0 _ _ a = return a
mhSteps n scoring proposer a = mhStep scoring proposer a >>= mhSteps (n - 1) scoring proposer

-- | A way to sample uniformly from floats in a given range
sampleFloatUniform :: Float -> Float ->  P Float
sampleFloatUniform l h = do
   u <- sample01
   return $ l + u * (h - l)
  
-- | Generate samples for a numeric type printvals given a scoring function
-- using metropolis hastings
mhFloat :: (Float, Float) -> (Float -> Float) -> P Float
mhFloat (lo, hi) scorer = mhSteps 10 scorer (const $ sampleFloatUniform lo hi) 0.5

Let's now check if this works! If it does work, then our histograms should look like the functions that we are plotting, since the function we pass to `mhFloat` is the probability density that we wish to sample from.

In [8]:
runsP 1000 (mhFloat (0, 6) (const 1.0)) >>= histogram "uniform" 20
runsP 1000 (mhFloat (0, 6) (^2)) >>= histogram "x^2" 20
runsP 1000 (mhFloat (0, 6) (abs . sin)) >>= histogram "|sin x|" 20
runsP 1000 (mhFloat (0, 6) (abs . cos)) >>= histogram "|cos x|" 20

uniform ▆▅▆▆▆▅▇█▅▅▆▆▇▆▆▄▆▅▆▇_

x^2 _____▁▁▁▂▂▂▂▃▃▄▅▅▆▆█▇

|sin x| _▂▄▅▆█▆▄▃▂_▁▃▆▅▅▆▆▃▂_

|cos x| █▇▅▄▁_▂▅▇▇▇▇▆▄▂_▁▂▆▆_

Great, all of them seem to work. Let's now use similar ideas to estimate the bias of a coin. The idea is as follows:

We have a model of a coin, and we know how likely heads or tails is given the model of a coin. Let's call this
$P(d|m)$ (probability of the $d$ata given the $m$odel). This is:

$$
\begin{align}
P(1|\text{bias}) &= \text{bias} \\
P(0|\text{bias}) &= 1 - \text{bias}
\end{align}
$$

What we want to do is the _inverse problem_.
Given observations about coin flips from a coin with an _unknown bias_, we wish to _predict its bias_.
That is, we want to find $P(\text{bias}|\text{data})$.

We solve this problem using Bayes' theorem. We know that

$$P(bias|data) = \frac{P(data|bias) P(bias)}{P(data)}$$ 

The denominator is normalzation factor that is constant for a fixed dataset. Thus, we write:

$$P(bias|data) \propto P(data|bias) P(bias)$$

This, if $P(bias)$ is our _prior belief_ about the biases, then $P(data|bias)$ is how much we need to multiply the
prior with to get the _posterios belief_.

In our case, as mentioned above, the value of $P(data|bias)$ is:

$$
\begin{align}
P(1|\text{bias}) &= \text{bias} \\
P(0|\text{bias}) &= 1 - \text{bias}
\end{align}
$$

In [9]:
-- | Given a list of observations from a coin and the bias, return a value proportional
-- to the coin having that bias. Find this by multiplying by bias if we have a 1, (1 - bias)
-- if we have a 0, for each heads/tails we see.
estimateBias :: [Int] -> Float -> Float
estimateBias obs bias = 
  product $ (map (\o -> if o == 1 then bias else (1 - bias)) obs)

replicateList :: Int -> [a] -> [a]
replicateList n as = mconcat $ replicate n as 

runsP 1000 (mhFloat (0, 1) $ estimateBias []) >>= histogram "estimate with no data" 20
runsP 1000 (mhFloat (0, 1) $ estimateBias [1]) >>= histogram "estimate with [1]" 20
runsP 1000 (mhFloat (0, 1) $ estimateBias [0]) >>= histogram "estimate with [0]" 20
runsP 1000 (mhFloat (0, 1) $ estimateBias [0, 1]) >>= histogram "estimate with [0, 1]" 20
runsP 1000 (mhFloat (0, 1) $ estimateBias [1, 0]) >>= histogram "estimate with [1, 0]" 20
runsP 1000 (mhFloat (0, 1) $ estimateBias [1, 0, 1, 0]) >>= histogram "estimate with [1, 0]x2" 20
runsP 1000 (mhFloat (0, 1) $ estimateBias (replicateList 8 [1, 0])) >>= histogram "estimate with [1, 0]x8" 20
runsP 1000 (mhFloat (0, 1) $ estimateBias (replicateList 20 [1, 0])) >>= histogram "estimate with [1, 0]x20" 20

estimate with no data ▇▄▅▇██▅▆▇▆▆▅▇▅▇▆▇▆▅▅_

estimate with [1] _▁▁▁▂▂▂▃▄▄▃▄▆▆▆▇▇▆▆█▇

estimate with [0] ▇█▇▆▆▅▅▅▄▄▂▃▃▂▂▁▁▁___

estimate with [0, 1] _▂▂▃▄▄▆▆▅█▇▇▅▇▅▄▃▃▂▁_

estimate with [1, 0] _▂▃▄▃▆▆▇▇▇▆▇█▇▆▅▅▃▃▁_

estimate with [1, 0]x2 __▁▂▄▄▅▅▇▆▇▇█▇▆▅▄▃▂▁_

estimate with [1, 0]x8 ____▁▂▃▄▄▆█▆▄▄▂▁____

estimate with [1, 0]x20 ____▁▁▂▃▃█▃▃▃▂▁____

Notice that the "estimate with no data" is roughly uniform: we knew nothing, so we assume that every event is equally likely.

as we gain more data, we update our beliefs about what the bias of the coin is. For example, if we see _just a single `[1]`_, we think it's much more likely for the coin to be biased. 

However, as soon as we see another data point as `[1, 0]`, we adjust our beliefs drastically: we now think the coin is most likely fair, since in two tosses of the coin, we saw once a heads and then a tails.

If one wishes to, we could link this update to notions of [maximum likelihood estimation](TODO: fill link), but we won't be pursuing this right now here. 

In this library, we have a duality of representations: For one, we are able to represent _random variables_ such as that of the biased coin. We think of `P Int` as a "random integer variable". On the other hand, for the bias estimation example, we live in the world of _probability distributions_: We think directly of the probabilities of the bias of the coin.

There are different ways of unifying this approach. One way of approaching this would be as the one [`monad-bayes`](TODO: fill link) takes, which is to add a `score` function into the `P` monad to multiply the distribution with some weight.  While this approach works, and we draw a lot of inspiration from their implementation, it still feels insufficient --- We should either live in the world of random variables, or in the world of distributions, and not mix these two worlds up. As Conal Elliot often says, "this lacks the feeling of _inevatibility_".  I would love to hear about library design ideas that manages to separate the two worlds cleanly!